Running boilerplate LDA models from gensim for topic modeling.
Inspired by: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [18]:
import nltk
import spacy
from nltk.corpus import stopwords
import pandas as pd
import gensim
from gensim.utils import simple_preprocess

# Offenseval Dataset

In [19]:
offenseval_train = pd.read_csv("./../../data/offenseval_train.csv")
offenseval_test = pd.read_csv("./../../data/offenseval_test.csv")
offenseval_val = pd.read_csv("./../../data/offenseval_val.csv")
# combine all three datasets
df = pd.concat([offenseval_train, offenseval_test, offenseval_val], ignore_index=True)

In [20]:
# load dataset and tokenize
df['tokenized'] = df['text'].apply(lambda x: simple_preprocess(x))
df.head()

,text,label,tokenized
0,@USER She is not leaving BB,0,"[user, she, is, not, leaving, bb]"
1,@USER @USER Ford and the conservatives hates t...,0,"[user, user, ford, and, the, conservatives, ha..."
2,@USER God is good to us :folded_hands: :thumbs...,0,"[user, god, is, good, to, us, folded_hands, th..."
3,@USER woman accusing Supreme Court nominee Kav...,0,"[user, woman, accusing, supreme, court, nomine..."
4,@USER CORRECTION: The Liberals won a false-maj...,0,"[user, correction, the, liberals, won, false, ..."


In [21]:
# combine all the tokenized texts into one list
all_tokenized_text = []
for i in range(len(df)):
    all_tokenized_text.extend(df.loc[i, 'tokenized'])


In [22]:
# build bigram and trigram models
bigram = gensim.models.Phrases(all_tokenized_text, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[all_tokenized_text], threshold=100)
bigram_maker = gensim.models.phrases.Phraser(bigram)
trigram_maker = gensim.models.phrases.Phraser(trigram)

In [23]:
# remove stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'user', 'url'])
all_tokenized_text = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in all_tokenized_text]

In [24]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
# form bigrams
all_tokenized_text_bi = [bigram_maker[doc] for doc in all_tokenized_text]
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
all_tokenized_text_bi = lemmatization(all_tokenized_text_bi)
# drop all the empty lists
all_tokenized_text_bi = [doc for doc in all_tokenized_text_bi if len(doc) > 0]

In [26]:
dictionary = gensim.corpora.Dictionary(all_tokenized_text_bi)
corpus = [dictionary.doc2bow(doc) for doc in all_tokenized_text_bi]

In [27]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=100, update_every=1, chunksize=100, passes=10, per_word_topics=True)

In [28]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda.print_topics(num_topics=10, num_words=10))

[(0,
  '0.490*"liberal" + 0.122*"come" + 0.000*"lie" + 0.000*"even" + 0.000*"year" '
  '+ 0.000*"shit" + 0.000*"thing" + 0.000*"talk" + 0.000*"really" + '
  '0.000*"love"'),
 (1,
  '0.258*"get" + 0.184*"people" + 0.133*"make" + 0.101*"need" + 0.095*"well" + '
  '0.000*"lie" + 0.000*"even" + 0.000*"year" + 0.000*"shit" + 0.000*"thing"'),
 (2,
  '0.383*"maga" + 0.110*"hate" + 0.024*"democratic" + 0.011*"mouth" + '
  '0.000*"cut" + 0.000*"lie" + 0.000*"even" + 0.000*"shit" + 0.000*"year" + '
  '0.000*"thing"'),
 (3,
  '0.219*"want" + 0.215*"right" + 0.103*"leave" + 0.045*"attack" + 0.000*"lie" '
  '+ 0.000*"even" + 0.000*"shit" + 0.000*"year" + 0.000*"thing" + '
  '0.000*"love"'),
 (4,
  '0.390*"say" + 0.132*"also" + 0.022*"assault" + 0.000*"lie" + 0.000*"even" + '
  '0.000*"shit" + 0.000*"year" + 0.000*"thing" + 0.000*"love" + 0.000*"time"'),
 (5,
  '0.124*"use" + 0.095*"white" + 0.094*"fuck" + 0.086*"tweet" + 0.053*"seem" + '
  '0.045*"court" + 0.028*"freedom" + 0.024*"close" + 0.017*"t

In [29]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=all_tokenized_text_bi, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score:', coherence_lda)

Coherence Score: 0.8521643028672626


UMass Measure: https://mimno.infosci.cornell.edu/papers/mimno-semantic-emnlp.pdf

In [30]:
import pickle 
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda, corpus, dictionary)
LDAvis_prepared


C:\Users\Kai\miniconda3\envs\cs6471\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.353702  0.056821       1        1  10.736349
8      0.017549 -0.355337       2        1  10.560113
9     -0.046985  0.040706       3        1  10.335590
0     -0.047064  0.037589       4        1   9.953969
6     -0.047936  0.038717       5        1   9.856474
3     -0.046353  0.036795       6        1   9.824051
7     -0.046585  0.037038       7        1   9.785466
2     -0.044979  0.035450       8        1   9.701857
5     -0.045984  0.036402       9        1   9.631581
4     -0.045365  0.035818      10        1   9.614549, topic_info=        Term         Freq        Total Category  logprob  loglift
40   liberal  6008.000000  6008.000000  Default  30.0000  30.0000
28       say  4620.000000  4620.000000  Default  29.0000  29.0000
38      maga  4576.000000  4576.000000  Default  28.0000  28.0000
69       gun  4464.000000  4464.000000  Default  27.0000  27.0000
125    think  3990.000000  3990.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
175      old     0.448866     3.715012  Topic10 -10.1801   0.2285
741     care     0.448764     3.714274  Topic10 -10.1803   0.2285
128      way     0.448662     3.713480  Topic10 -10.1806   0.2284
439      bad     0.448977     3.716124  Topic10 -10.1799   0.2284
874    first     0.448327     3.711117  Topic10 -10.1813   0.2283

[904 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
78       10  0.997905          also
22       10  0.987026       assault
8         6  0.993723        attack
43        7  0.998514          back
45        9  0.987849         close
84        4  0.997742          come
2         2  0.998805  conservative
70        3  0.999166       control
18        9  0.994030         court
62        8  0.644325           cut
5         8  0.987888    democratic
33        3  0.995657          free
6         9  0.988961       freedom
61        9  0.997392          fuck
112       1  0.999002           get
32        2  0.998924            go
13        5  0.998905          good
69        3  0.999156           gun
3         8  0.997804          hate
60        2  0.998138          know
0         6  0.997591         leave
40        4  0.999383       liberal
71        5  0.994072           lol
44        5  0.988906           lot
38        8  0.999248          maga
101       1  0.997844          make
74        8  0.977643         mouth
68        1  0.997743          need
254       1  0.998538        people
79        7  0.996564         point
10        6  0.998750         right
28       10  0.999229           say
57        2  0.997752           see
49        9  0.994909          seem
36        9  0.982377          tcot
125       5  0.999258         think
170       7  0.998808         trump
56        9  0.996940         tweet
54        9  0.998012           use
35        6  0.998941          want
104       1  0.997515          well
75        9  0.997458         white
16        7  0.997967         woman, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 10, 1, 7, 4, 8, 3, 6, 5])